In [1]:
import dolfin as dl
import hippylib as hl
import matplotlib.pyplot as plt
import numpy as np

from spin.core import problem
from spin.fenics import converter as fex_converter
from spin.hippylib import hessian as hlx_hessian
from spin.hippylib import misfit as hlx_misfit
from spin.hippylib import optimization as hlx_optimization
from spin.hippylib import prior as hlx_prior

In [2]:
mesh = dl.IntervalMesh(10, -1, 1)
problem_settings = problem.SPINProblemSettings(
    mesh=mesh,
    pde_type="mean_exit_time",
    inference_type="drift_only",
    drift=(("-x[0]",)),
    log_squared_diffusion=("std::log(1.0)",),
)
problem_builder = problem.SPINProblemBuilder(problem_settings)
spin_problem = problem_builder.build()

In [3]:
prior_settings = hlx_prior.PriorSettings(
    function_space=spin_problem.function_space_parameters,
    mean=("-0.5*x[0]",),
    variance=("0.1",),
    correlation_length=("0.1",),
)
prior_builder = hlx_prior.BilaplacianVectorPriorBuilder(prior_settings)
spin_prior = prior_builder.build()

In [4]:
observation_points = np.array((-0.8, -0.6, -0.4, -0.2, 0.0, 0.2, 0.4, 0.6, 0.8))
exact_solution = np.array(
    (
        0.21648217,
        0.42039515,
        0.58953849,
        0.70196354,
        0.74143722,
        0.70196354,
        0.58953849,
        0.42039515,
        0.21648217,
    )
)
perturbed_solution = exact_solution + np.random.normal(0, 0.01, exact_solution.shape)
noise_variance = 0.01 * np.ones(exact_solution.shape)

misfit_settings = hlx_misfit.MisfitSettings(
    function_space=spin_problem.function_space_variables,
    observation_points=observation_points,
    observation_values=perturbed_solution,
    noise_variance=noise_variance,
)
misfit_builder = hlx_misfit.MisfitBuilder(misfit_settings)
spin_misfit = misfit_builder.build()

In [5]:
inference_model = hl.Model(
    spin_problem.hippylib_variational_problem, spin_prior.hippylib_prior, spin_misfit
)

In [ ]:
optimization_settings = hlx_optimization.SolverSettings(
    relative_tolerance=1e-3, absolute_tolerance=1e-3, verbose=True
)
initial_guess = spin_prior.mean_array
newton_solver = hlx_optimization.NewtonCGSolver(optimization_settings, inference_model)
solver_solution = newton_solver.solve(initial_guess)